# Sentence Classification with Pre-Trained Embeddings  

### Introduction

Embeddings are a way to represent words (or more generally, *tokens*) as vectors. These vectors are useful for many tasks in NLP, including but not limited to: Text Generation, Machine Translation, and Sentence Classification. In this notebook, we will be exploring the concept of Embeddings, and using them for Sentence Classification.

In [3]:
import re
!pip install gpt4all

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

from gpt4all import Embed4All

import torch
import torch.nn as nn

nltk.download('stopwords')
nltk.download('wordnet')

  Obtaining dependency information for gpt4all from https://files.pythonhosted.org/packages/b3/5b/1f4334f54546605efb878129ecde1a6bbab1710c233440fef2888b56b5c8/gpt4all-2.0.2-py3-none-macosx_10_15_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.3 MB/s eta 0:00:0000:0100:010m


[nltk_data] Downloading package stopwords to /Users/ajm/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/ajm/nltk_data...


True

## Exploring Embeddings [10 Marks]

Put simply, Embeddings are fixed-size **dense** vector representations of tokens in natural language. This means you can represent words as vectors, sentences as vectors, even other entities like entire graphs as vectors.

So what really makes them different from something like One-Hot vectors?

What's special is that they have semantic meaning baked into them. This means you can model relationships between entities in text, which itself leads to a lot of fun applications. All modern architectures make use of Embeddings in some way.

You can read more about them [here](https://aman.ai/primers/ai/word-vectors/).

We will be using *pretrained* Embeddings: this means that we will be using Embeddings that have already been trained on a large corpus of text. This is because training Embeddings from scratch is a very computationally expensive task, and we don't have the resources to do so. Fortunately, there were some good samaritans who have already done this for us, and we can use their publicly available Embeddings for our own tasks.


We will load in pretrained Embeddings here and examine some of their properties. If you're interested, feel free look up the [Word2Vec model](https://arxiv.org/abs/1301.3781): this is the model that was trained to give us the embeddings you will see below.

In [4]:
# Download the pretrained word2vec model (this may take a few minutes)
corpus = api.load('text8')
w2vmodel = Word2Vec(corpus)

print("Done loading word2vec model!")

[==================================================] 100.0% 31.6/31.6MB downloaded
Done loading word2vec model!


Now that we've loaded in the Embeddings, we can create an Embedding **layer** in PyTorch, `nn.Embedding`, that will perform the processing step for us.

Note in the following cell how there is a given **vocab size** and **embedding dimension** for the model: this is important to note because some sets of Embeddings may be defined for a large set of words (a large vocab), whereas older ones perhaps have a smaller set (a small vocab); the Embedding dimension essentially tells us how many *features* have been learned for a given word, that will allow us to perform further processing on top of.

In [5]:
# Define embedding layer using gensim
embedding_layer = nn.Embedding.from_pretrained(torch.FloatTensor(w2vmodel.wv.vectors))

# Get some information from the w2vmodel
print(f"Vocab size: {len(w2vmodel.wv.key_to_index)}")

print(f"Some of the words in the vocabulary:\n{list(w2vmodel.wv.key_to_index.keys())[:10]}")

print(f"Embedding dimension: {w2vmodel.wv.vectors.shape[1]}")

Vocab size: 71290
Some of the words in the vocabulary:
['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two']
Embedding dimension: 100


Now, for a demonstration, we instantiate two words, turn them into numbers (encoding them via their index in the vocab), and pass them through the Embedding layer. 

Note how the resultant Embeddings both have the same shape: 1 word, and 100 elements in the vector.

In [6]:
# Take two words and get their embeddings
word1 = "king"
word2 = "queen"

def word2vec(word):
    return embedding_layer(torch.LongTensor([w2vmodel.wv.key_to_index[word]]))

king_embedding = word2vec(word1)
queen_embedding = word2vec(word2)

print(f"Embedding Shape for '{word1}': {king_embedding.shape}")
print(f"Embedding Shape for '{word2}': {queen_embedding.shape}")

Embedding Shape for 'king': torch.Size([1, 100])
Embedding Shape for 'queen': torch.Size([1, 100])


When we have vectors whose scale is arbitrary, one nice way to measure how *similar* they are is with the Cosine Similarity measure.


$$ \text{Cosine Similarity}(\mathbf{u},\mathbf{v}) = \frac{\mathbf{u} \cdot \mathbf{v}}{\|\mathbf{u}\| \|\mathbf{v}\|} $$


We can apply this idea to our Embeddings. To see how "similar" two words are to the model, we can generate their Embeddings and take the Cosine Similarity of them. This will be a number between -1 and 1 (just like the range of the cosine function). When the number is close to 0, the words are not similar.

In [7]:
def cosine_similarity(vec1, vec2):
    '''
    Computes the cosine similarity between two vectors
    '''

    #   Compute the cosine similarity between the two vectors (using PyTorch)
    return torch.nn.functional.cosine_similarity(vec1, vec2, dim=0)

def compute_word_similarity(word1, word2):
    '''
    Takes in two words, computes their embeddings and returns the cosine similarity
    '''
    
    v1 = embedding_layer(torch.LongTensor([w2vmodel.wv.key_to_index[word1]])).view(-1)
    v2 = embedding_layer(torch.LongTensor([w2vmodel.wv.key_to_index[word2]])).view(-1)
    
    return cosine_similarity(v1, v2)

#   Define three words (one pair should be similar and one pair should be dissimilar) and compute their similarity
word1 = "king"
word2 = "queen"
word3 = "apple"
print(f"Similarity between '{word1}' and '{word2}': {compute_word_similarity(word1, word2)}")
print(f"Similarity between '{word1}' and '{word3}': {compute_word_similarity(word1, word3)}")

Similarity between 'king' and 'queen': 0.6975234746932983
Similarity between 'king' and 'apple': 0.1903911828994751


In [8]:
# Run this cell if you're done with the above section
del embedding_layer

## Sentence Classification Classification with Sentence Embeddings 

Now let's move on to an actual application: classifying whether a tweet is about a real disaster or not. As you can imagine, this could be a valuable model when monitoring social media for disaster relief efforts.

Since we are using Sentence Embeddings, we want something that will take in a sequence of words and throw out a single fixed-size vector. For this task, we will make use of an LLM via the `gpt4all` library.

This library will allow us to generate pretrained embeddings for sentences, that we can use as **features** to feed to any classifier of our choice.

In [9]:
# Read in the data here
df = pd.read_csv("./disaster_tweets.csv")
df = df[["text", "target"]]

# Split the data
train, val = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape, val.shape)

(6090, 2) (1523, 2)


Before jumping straight to Embeddings, since our data is sourced from the cesspool that is Twitter, we should probably do some cleaning. This can involve the removal of URLs, punctuation, numbers that don't provide any meaning, stopwords, and so on.'

In the following cell, write functions to clean the sentences. You are allowed to add more functions if you wish, but the ones provided are the bare minimum.

**Note:** After cleaning your sentences, it is possible that you may end up with empty sentences (or some that are so short they have lost all meaning). In this event, since we want to demonstrate setting up a Sentence Classification task, you should remove them from your dataset (data cleaning is not the center of this notebook).

In [11]:
#  Clean the sentences (5 marks)
import string

#  Fill out the following functions, adding more if desired

def lowercase(txt):
    
    return txt.lower()

def remove_punctuation(txt):
    
    return txt.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(txt):
    
    stop_words = set(stopwords.words('english'))
    words = txt.split()
    filtered_words = ' '.join([word for word in words if word not in stop_words])
    
    return filtered_words

def remove_numbers(txt):
    
    return re.sub(r'\d+', '', txt)

def remove_url(txt):
    
    return re.sub(r'http\S+|www\S+|https\S+', '', txt, flags=re.MULTILINE)

def normalize_sentence(txt):
    '''
    Aggregates all the above functions to normalize/clean a sentence
    '''
    
    txt = lowercase(txt)
    txt = remove_url(txt)
    txt = remove_punctuation(txt)
    txt = remove_numbers(txt)
    txt = remove_stopwords(txt)
    
    return txt

# Clean the sentences
df['cleaned_text'] = df['text'].apply(normalize_sentence)

# Filter sentences that are too short (less than 20ish characters)
df = df[df['cleaned_text'].str.len() >= 20]

Now for the fun part, creating our Embeddings!

We will be using the `gpt4all.Embed4All` class for this purpose. You can look up the documentation [here](https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All.embed).

This functionality makes use of a model called [Sentence-BERT](https://arxiv.org/abs/1908.10084). This is a Transformer-based model that has been trained on a large corpus of text, and is able to generate high-quality Sentence Embeddings for us.

In [18]:
# Generate embeddings for train and validation sentences (5 marks)

feature_extractor = Embed4All()

# Encode the train samples
def encode(sentences, feature_extractor):

    embeddings = []

    for sentence in sentences:

        embeddings.append(feature_extractor.embed(sentence))

    return embeddings

# Encode the train sentences
train_embeddings = encode(train['text'].tolist(), feature_extractor)

# Encode the validation sentences
val_embeddings = encode(val['text'].tolist(), feature_extractor)

# Ready the labels
train_labels = train['target'].values
val_labels = val['target'].values


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


Now with our Embeddings ready, we can move on to the actual classification task.

You have the choice of using **any** classifier you wish. You can use a simple Logistic Regression model, get fancy with Support Vector Machines, or even use a Neural Network. The choice is yours.

We will be looking for a model with a **Validation Accuracy** of around $0.8$. You must also use this model to make predictions on your own provided inputs, after completing the `predict` function.

In [19]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression()

model.fit(train_embeddings, train_labels)

val_preds = model.predict(val_embeddings)

val_accuracy = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.8181221273801708


In [30]:

def predict(sentence, clf):
    '''
    Takes in a sentence and returns the predicted class along with the probability
    '''
    #   Clean and encode the sentence
    sentence = normalize_sentence(sentence)
    embedding = encode([sentence], feature_extractor)[0]

    #   Predict the class and probability
    prediction = clf.predict([embedding])
    probability = clf.predict_proba([embedding])

    return prediction[0], probability[0]
    
#   Predict on a few of your own sentences

sentence = "There was a massive earthquake in Miami that killed 10000 people."
predicted_class, probabilities = predict(sentence, model)
print(f"Predicted class: {predicted_class}, Probabilities: {probabilities}")

Predicted class: 1, Probabilities: [0.10395519 0.89604481]
